In [5]:
from graphviz import Digraph
import pandas as pd # Lectura de csv

arbol = pd.read_csv('arbol.csv')

print(arbol.head())
print(arbol.columns.values)
ancestros = arbol.loc[arbol['relacion'] == 'primerancestro', 'persona1'].iloc[0]
arbol['registro_persona'] = 0    # Indicador para saber que la persona a sido registrada en el arbol

incomp = [ancestros]
comp = []

diagrama = Digraph(comment = 'Ancestry', graph_attr = {'splines':'ortho'})
nodos = []

# Inicialización del nodo

det = str(arbol.loc[arbol['persona1'] == ancestros, 'detalles'][0])
g = arbol.loc[arbol['persona1'] == ancestros, 'genero'][0]
forma = 'rect' if g == 'M' else 'ellipse' # En caso de que sea masculino el nodo será rectangular, de lo contrario sera un elipse
diagrama.node(ancestros, ancestros, tooltip = det, shape = forma)
nodos.append(ancestros)

arbol.loc[arbol['persona1'] == ancestros, 'registro_persona'] = 1

max_iter = 10 # Hace referencia a las generaciones que contiene el arbol

for i in range(0, max_iter):
    print(i)
    temp = arbol[arbol['registro_persona'] == 0]

    if len(temp) == 0:      # Rompe el loop al registrar a todos las personas
        break
    else:
        temp['generacion_per'] = temp.apply(lambda x: 1 if x['persona2'] in incomp else 0, axis = 1)

        # Relación conyuge
        generacion = temp[(temp['generacion_per'] == 1) & (temp['relacion'] == 'conyuge')]
        if len(generacion) > 0:
            for j in range(0, len(generacion)):
                per1 = generacion['persona1'].iloc[j]
                per2 = generacion['persona2'].iloc[j]
                det = str(generacion['detalles'].iloc[j])
                g = generacion['genero'].iloc[j]
                forma = 'rect' if g == 'm' else 'ellipse'
                with diagrama.subgraph() as subs:
                    subs.attr(rank = 'same')
                    subs.node(per1, per1, tooltip = det, shape = forma, fillcolor = "red")
                    nodos.append(per1)
                    subs.edge(per2, per1, arrowhead = 'none', color = "black:pink:black")

        # Relación hijos
        generacion = temp[(temp['generacion_per'] == 1) & (temp['relacion'] == 'hijo')]
        if len(generacion) > 0:
            for j in range(0, len(generacion)):
                per1 = generacion['persona1'].iloc[j]
                per2 = generacion['persona2'].iloc[j]
                det = str(generacion['detalles'].iloc[j])
                g = generacion['genero'].iloc[j]
                forma = 'rect' if g == 'm' else 'ellipse'
                diagrama.node(per1, per1, tooltip = det, shape = forma)
                nodos.append(per1)
                diagrama.edge(per2, per1)

        comp.extend(incomp)
        incomp = list(temp.loc[temp['generacion_per'] == 1, 'persona1'])
        arbol['registro_persona'] = temp.apply(lambda x: 1 if (x['persona1'] in incomp) | (x['persona1'] in comp) else 0, axis = 1)

diagrama.format = 'svg'
diagrama.render('ArbolGenealogico.gv.svg', view = True)

      persona1        relacion     persona2 genero  detalles
0     jorgeIII  primerancestro          NaN      m         1
1  guillermoIV            hijo     jorgeIII      m         2
2     victoria            hijo  guillermoIV      f         3
3   eduardoVII            hijo     victoria      f         4
4       jorgeV            hijo   eduardoVII      m         5
['persona1' 'relacion' 'persona2' 'genero' 'detalles']
0
1
2
3
4
5
6
7
8
9


<ipython-input-5-62b25224394f>:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['generacion_per'] = temp.apply(lambda x: 1 if x['persona2'] in incomp else 0, axis = 1)


'ArbolGenealogico.gv.svg.svg'